In [1]:
!pip install transformers
!pip install gradio

     |████████████████████████████████| 2.9 MB 4.4 MB/s 
     |████████████████████████████████| 3.3 MB 16.2 MB/s 
     |████████████████████████████████| 56 kB 2.3 MB/s 
     |████████████████████████████████| 895 kB 14.3 MB/s 
     |████████████████████████████████| 596 kB 34.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 3.6 MB 5.2 MB/s 
     |████████████████████████████████| 206 kB 52.0 MB/s 
     |████████████████████████████████| 1.9 MB 56.2 MB/s 
     |████████████████████████████████| 3.5 MB 39.9 MB/s 
     |████████████████████████████████| 961 kB 57.1 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
  Created wheel for ffmpy: filename=ffmpy-0.3.0-py3-none-any.whl size=4710 sha256=42accef86bef6b0102cdbc447b0dee3c5949307808fc3d0de010c6028ecab31a
  Stored in directory: /root/.cache/pip/wheels/13/e4/6c/e8059816e86796a5

In [2]:
import numpy as np
import tensorflow as tf
from tensorboard import notebook
from transformers import DistilBertTokenizerFast, TFDistilBertForQuestionAnswering

import gradio as gr

In [3]:
!mkdir squad
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json -O squad/train-v2.0.json
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json -O squad/dev-v2.0.json

--2021-10-20 04:39:16--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘squad/train-v2.0.json’

squad/train-v2.0.js 100%[===================>]  40.17M   122MB/s    in 0.3s    

2021-10-20 04:39:17 (122 MB/s) - ‘squad/train-v2.0.json’ saved [42123633/42123633]

--2021-10-20 04:39:17--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘squad/dev-v2.0.json’

squ

In [4]:
import json
from pathlib import Path

def read_squad(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)

    return contexts, questions, answers

train_contexts, train_questions, train_answers = read_squad('squad/train-v2.0.json')
val_contexts, val_questions, val_answers = read_squad('squad/dev-v2.0.json')

In [5]:
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)

        # sometimes squad answers are off by a character or two – fix this
        if context[start_idx:end_idx] == gold_text:
            answer['answer_end'] = end_idx
        elif context[start_idx-1:end_idx-1] == gold_text:
            answer['answer_start'] = start_idx - 1
            answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
        elif context[start_idx-2:end_idx-2] == gold_text:
            answer['answer_start'] = start_idx - 2
            answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters

add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)

In [6]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [7]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))

        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length

    encodings.update({'start_logits': start_positions, 'end_logits': end_positions})

add_token_positions(train_encodings, train_answers)
add_token_positions(val_encodings, val_answers)

In [8]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    {key: train_encodings[key] for key in ['input_ids', 'attention_mask']},
    {key: train_encodings[key] for key in ['start_logits', 'end_logits']}
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    {key: val_encodings[key] for key in ['input_ids', 'attention_mask']},
    {key: val_encodings[key] for key in ['start_logits', 'end_logits']}
))

In [9]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
model = TFDistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased")
model.distilbert.return_dict = False
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

BATCH_SIZE = 2
history = model.fit(train_dataset.take(100).batch(BATCH_SIZE),
                    validation_data=val_dataset.take(10).batch(BATCH_SIZE),
                    epochs=3, 
                    batch_size=BATCH_SIZE, 
                    callbacks=[tensorboard_callback],
                    verbose=1)  

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForQuestionAnswering: ['activation_13', 'vocab_transform', 'vocab_projector', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_19', 'qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
50/50 [==============================] - 372s 7s/step - loss: 10.2183 - end_logits_loss: 5.0786 - start_logits_loss: 5.1397 - end_logits_accuracy: 0.0300 - start_logits_accuracy: 0.0000e+00 - val_loss: 9.1735 - val_end_logits_loss: 4.6620 - val_start_logits_loss: 4.5115 - val_end_logits_accuracy: 0.0000e+00 - val_start_logits_accuracy: 0.0000e+00
Epoch 2/3
50/50 [==============================] - 351s 7s/step - loss: 7.8124 - end_logits_loss: 3.8853 - start_logits_loss: 3.9271 - end_logits_accuracy: 0.1400 - start_logits_accuracy: 0.0900 - val_loss: 7.9113 - val_end_logits_loss: 3.8520 - val_start_logits_loss: 4.0592 - val_end_logits_accuracy: 0.0000e+00 - val_start_logits_accuracy: 0.0000e+00
Epoch 3/3
50/50 [==============================] - 353s 7s/step - loss: 6.2014 - end_logits_loss: 3.1165 - start_logits_loss: 3.0849 - end_logits_accuracy: 0.1000 - start_logits_accuracy: 0.1600 - val_loss: 7.5103 - val_end_logits_loss: 4.0385 - val_start_logits_loss: 3.4718 - val_end_l

In [16]:
def predict(context, question):
  input_encodings = tokenizer(context, question, return_tensors='tf', truncation=True, padding=True)
  input_dict = tf.data.Dataset.from_tensor_slices((
      {key: input_encodings[key] for key in ['input_ids', 'attention_mask']}
  ))

  outputs = model.predict(input_dict)
  start_logits = outputs.start_logits
  end_logits = outputs.end_logits

  all_tokens = tokenizer.convert_ids_to_tokens(input_encodings["input_ids"].numpy()[0])
  answer = ' '.join(all_tokens[tf.math.argmax(start_logits, 0)[0] : tf.math.argmax(end_logits, 0)[0]+1])
  return answer

iface = gr.Interface(fn=predict, inputs=["text", "text"], outputs=["text"])
iface.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
This share link will expire in 72 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted
Running on External URL: https://57992.gradio.app


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7862/',
 'https://57992.gradio.app')